Clean Landsat composites created with the file "download_landsat_composite.ipynb"

1. remove high values (1.000000e+20) and replace with NAN
2. add spectral indices

In [ ]:
# Import general libraries
import geopandas as gpd
import rasterio
from rasterio.transform import from_origin
import numpy as np
import rasterio

from pathlib import Path
from matplotlib import pyplot as plt

# Import EOdal libraries
from eodal.core.band import Band
from eodal.core.raster import RasterCollection

# define the year/years for the download
years = list(range(1984, 2023)) # from to
excluded_years = [1988, 1992, 2023] #no scenes
filtered_years = [year for year in years if year not in excluded_years]

for year in filtered_years: 

     # Path to the input TIFF file
    input_tif_path = f'S:\MSc_23_TimckeFinn\data\EOdal\landsat_median_composite_{year}.tif'

    # Path to the output TIFF file
    output_tif_path = f'S:\MSc_23_TimckeFinn\data\EOdal\landsat_median_composite_{year}_clean.tif'

    # Open the TIFF file using rasterio
    with rasterio.open(input_tif_path) as src:

        # Read all bands of the raster data
        raster_data = src.read()

        # Replace specific values (e.g., 1e+20) with NaN for each band
        replaced_data = np.where(raster_data == 1e+20, np.nan, raster_data)

        # Create a new raster file with NaN values
        with rasterio.open(
            output_tif_path,
            'w',
            driver='GTiff',
            height=src.height,
            width=src.width,
            count=src.count,  # Number of bands
            dtype=replaced_data.dtype,
            crs=src.crs,
            transform=src.transform,
        ) as dst:
            dst.write(replaced_data)

    #read tif from disk:
    fpath_raster = Path(f'S:\MSc_23_TimckeFinn\data\EOdal\landsat_median_composite_{year}_clean.tif')

    # initialize a RasterCollection from the .tif file
    composite = RasterCollection.from_multi_band_raster(fpath_raster=fpath_raster,
                                                        band_names_dst= ["blue_median", "green_median", "red_median", "nir08_median", "swir16_median","swir22_median"],
                                                        band_aliases = ["blue", "green", "red", "nir_1", "swir_1","swir_2"])

    # calculate some SI
    composite.calc_si('NDVI', inplace = True)
    composite.calc_si('NDWI', inplace = True)
    composite.calc_si('EVI', inplace = True)
    composite.calc_si('CI_GREEN', inplace = True)
    composite.calc_si('MSAVI', inplace = True)
    composite.calc_si('BSI', inplace = True)

    # save as GeoTiff
    composite.to_rasterio(f'S:\MSc_23_TimckeFinn\data\EOdal\landsat_median_composite_{year}_clean_si.tif')

    # print band summaries
    composite.band_summaries()


In [ ]:
""" #plot band combinations

fig, axs = plt.subplots(1, 3, figsize = (20, 10), sharey=True)

composite.plot_multiple_bands(["nir08_median", "red_median", "green_median"], ax=axs[0])
axs[0].set_title("Color Infrared")
# This band combination is also called the near-infrared (NIR) composite. It uses near-infrared (5), red (4), and green (3). Because chlorophyll reflects near-infrared light, this band composition is useful for analyzing vegetation. In particular, areas in red have better vegetation health. Dark areas are water and urban areas are white.

composite.plot_multiple_bands(["swir16_median", "nir08_median", "blue_median"], ax=axs[1])
axs[1].set_title("Agriculture")
# This band combination uses SWIR-1 (6), near-infrared (5), and blue (2). It’s commonly used for crop monitoring because of the use of short-wave and near-infrared. Healthy vegetation appears dark green. But bare earth has a magenta hue.

composite.plot_multiple_bands(["swir22_median", "swir16_median", "red_median"], ax=axs[2])
axs[2].set_title("Short-Wave Infrared")

# The short-wave infrared band combination uses SWIR-2 (7), SWIR-1 (6), and red (4). This composite displays vegetation in shades of green. While darker shades of green indicate denser vegetation, sparse vegetation has lighter shades. Urban areas are blue and soils have various shades of brown.
#plot individual bands:
for band in composite.band_names:
    fig, axs = plt.subplots(1, 2, figsize = (20, 10), sharey=True)
    composite.plot_multiple_bands(["red", "green", "blue"], ax=axs[0])
    axs[0].set_title("RGB")
    if band == 'blue_median':
        composite.plot_band(band, colormap="Blues", ax=axs[1])
        axs[1].set_title(band)
    if band == 'green_median':
        composite.plot_band(band, colormap="Greens", ax=axs[1])
        axs[1].set_title(band)
    if band == 'red_median':
        composite.plot_band(band, colormap="Reds", ax=axs[1])
        axs[1].set_title(band)
    if band == 'nir08_median':
        composite.plot_band(band, colormap="Greys_r", ax=axs[1])
        axs[1].set_title(band)
    if band == 'swir16_median':
        composite.plot_band(band, colormap="Greys_r", ax=axs[1])
        axs[1].set_title(band)
    if band == 'swir22_median':
        composite.plot_band(band, colormap="Greys_r", ax=axs[1])
        axs[1].set_title(band)
    if band == 'NDVI':
        composite.plot_band(band, colormap="YlGn", ax=axs[1])
        axs[1].set_title(band)
    if band == 'NDWI':
        composite.plot_band(band, colormap="GnBu", ax=axs[1])
        axs[1].set_title(band)
    if band == 'EVI':
        composite.plot_band(band, colormap="YlGn", ax=axs[1])
        axs[1].set_title(band)
    if band == 'CI_GREEN':
        composite.plot_band(band, colormap="YlGn", ax=axs[1])
        axs[1].set_title(band)
    if band == 'MSAVI':
        composite.plot_band(band, colormap="YlGn", ax=axs[1])
        axs[1].set_title(band)
    if band == 'BSI':
        composite.plot_band(band, colormap="pink_r", ax=axs[1])
        axs[1].set_title(band)
 """